# Project Work Part 1

#### a)
Downloading data and displaying some of it

In [156]:
import numpy as np
import polars as pl

# read data
links_df = pl.read_csv("../data/links.csv", schema={"movieId": pl.Int32, "imdb": pl.Int32, "tmdbId": pl.Int32})
movies_df = pl.read_csv("../data/movies.csv", schema={"movieId": pl.Int32, "title": pl.String, "genres": pl.String}).with_row_index()
ratings_df = pl.read_csv("../data/ratings.csv", schema={"userId": pl.Int32, "movieId": pl.Int32, "rating": pl.Float32, "timestamp": pl.Int32})
tags_df = pl.read_csv("../data/tags.csv", schema={"userId": pl.Int32, "movieId": pl.Int32, "tag": pl.String, "timestamp": pl.Int32})

print("Number of ratings: ", ratings_df.height)

Number of ratings:  100836


In [157]:
print(links_df)

shape: (9_742, 3)
┌─────────┬─────────┬────────┐
│ movieId ┆ imdb    ┆ tmdbId │
│ ---     ┆ ---     ┆ ---    │
│ i32     ┆ i32     ┆ i32    │
╞═════════╪═════════╪════════╡
│ 1       ┆ 114709  ┆ 862    │
│ 2       ┆ 113497  ┆ 8844   │
│ 3       ┆ 113228  ┆ 15602  │
│ 4       ┆ 114885  ┆ 31357  │
│ 5       ┆ 113041  ┆ 11862  │
│ …       ┆ …       ┆ …      │
│ 193581  ┆ 5476944 ┆ 432131 │
│ 193583  ┆ 5914996 ┆ 445030 │
│ 193585  ┆ 6397426 ┆ 479308 │
│ 193587  ┆ 8391976 ┆ 483455 │
│ 193609  ┆ 101726  ┆ 37891  │
└─────────┴─────────┴────────┘


In [158]:
print(movies_df)

shape: (9_742, 4)
┌───────┬─────────┬─────────────────────────────────┬─────────────────────────────────┐
│ index ┆ movieId ┆ title                           ┆ genres                          │
│ ---   ┆ ---     ┆ ---                             ┆ ---                             │
│ u32   ┆ i32     ┆ str                             ┆ str                             │
╞═══════╪═════════╪═════════════════════════════════╪═════════════════════════════════╡
│ 0     ┆ 1       ┆ Toy Story (1995)                ┆ Adventure|Animation|Children|C… │
│ 1     ┆ 2       ┆ Jumanji (1995)                  ┆ Adventure|Children|Fantasy      │
│ 2     ┆ 3       ┆ Grumpier Old Men (1995)         ┆ Comedy|Romance                  │
│ 3     ┆ 4       ┆ Waiting to Exhale (1995)        ┆ Comedy|Drama|Romance            │
│ 4     ┆ 5       ┆ Father of the Bride Part II (1… ┆ Comedy                          │
│ …     ┆ …       ┆ …                               ┆ …                               │
│ 9737  ┆ 1935

In [159]:
print(ratings_df)

shape: (100_836, 4)
┌────────┬─────────┬────────┬────────────┐
│ userId ┆ movieId ┆ rating ┆ timestamp  │
│ ---    ┆ ---     ┆ ---    ┆ ---        │
│ i32    ┆ i32     ┆ f32    ┆ i32        │
╞════════╪═════════╪════════╪════════════╡
│ 1      ┆ 1       ┆ 4.0    ┆ 964982703  │
│ 1      ┆ 3       ┆ 4.0    ┆ 964981247  │
│ 1      ┆ 6       ┆ 4.0    ┆ 964982224  │
│ 1      ┆ 47      ┆ 5.0    ┆ 964983815  │
│ 1      ┆ 50      ┆ 5.0    ┆ 964982931  │
│ …      ┆ …       ┆ …      ┆ …          │
│ 610    ┆ 166534  ┆ 4.0    ┆ 1493848402 │
│ 610    ┆ 168248  ┆ 5.0    ┆ 1493850091 │
│ 610    ┆ 168250  ┆ 5.0    ┆ 1494273047 │
│ 610    ┆ 168252  ┆ 5.0    ┆ 1493846352 │
│ 610    ┆ 170875  ┆ 3.0    ┆ 1493846415 │
└────────┴─────────┴────────┴────────────┘


In [160]:
print(tags_df)

shape: (3_683, 4)
┌────────┬─────────┬──────────────────┬────────────┐
│ userId ┆ movieId ┆ tag              ┆ timestamp  │
│ ---    ┆ ---     ┆ ---              ┆ ---        │
│ i32    ┆ i32     ┆ str              ┆ i32        │
╞════════╪═════════╪══════════════════╪════════════╡
│ 2      ┆ 60756   ┆ funny            ┆ 1445714994 │
│ 2      ┆ 60756   ┆ Highly quotable  ┆ 1445714996 │
│ 2      ┆ 60756   ┆ will ferrell     ┆ 1445714992 │
│ 2      ┆ 89774   ┆ Boxing story     ┆ 1445715207 │
│ 2      ┆ 89774   ┆ MMA              ┆ 1445715200 │
│ …      ┆ …       ┆ …                ┆ …          │
│ 606    ┆ 7382    ┆ for katie        ┆ 1171234019 │
│ 606    ┆ 7936    ┆ austere          ┆ 1173392334 │
│ 610    ┆ 3265    ┆ gun fu           ┆ 1493843984 │
│ 610    ┆ 3265    ┆ heroic bloodshed ┆ 1493843978 │
│ 610    ┆ 168248  ┆ Heroic Bloodshed ┆ 1493844270 │
└────────┴─────────┴──────────────────┴────────────┘


#### b)
User-based collaborative filtering using the Pearson correlation function

In [161]:
# The pearson correlation function is found in scipy package
import scipy as sp

def get_pearson_correlation(a, b):
    return sp.stats.pearsonr(a, b)[0]

Let's calculate person correlation between userId 1 and all other users

In [162]:
# Create a ratings matrix by pivoting the ratings dataframe, filling null values with 0
ratings_matrix = (
    ratings_df
    .drop("timestamp")
    .sort("movieId")
    .pivot("movieId", index="userId")
    .sort("userId")
    .fill_null(strategy="zero")
)
print(ratings_matrix)

shape: (610, 9_725)
┌────────┬─────┬─────┬─────┬───┬────────┬────────┬────────┬────────┐
│ userId ┆ 1   ┆ 2   ┆ 3   ┆ … ┆ 193583 ┆ 193585 ┆ 193587 ┆ 193609 │
│ ---    ┆ --- ┆ --- ┆ --- ┆   ┆ ---    ┆ ---    ┆ ---    ┆ ---    │
│ i32    ┆ f32 ┆ f32 ┆ f32 ┆   ┆ f32    ┆ f32    ┆ f32    ┆ f32    │
╞════════╪═════╪═════╪═════╪═══╪════════╪════════╪════════╪════════╡
│ 1      ┆ 4.0 ┆ 0.0 ┆ 4.0 ┆ … ┆ 0.0    ┆ 0.0    ┆ 0.0    ┆ 0.0    │
│ 2      ┆ 0.0 ┆ 0.0 ┆ 0.0 ┆ … ┆ 0.0    ┆ 0.0    ┆ 0.0    ┆ 0.0    │
│ 3      ┆ 0.0 ┆ 0.0 ┆ 0.0 ┆ … ┆ 0.0    ┆ 0.0    ┆ 0.0    ┆ 0.0    │
│ 4      ┆ 0.0 ┆ 0.0 ┆ 0.0 ┆ … ┆ 0.0    ┆ 0.0    ┆ 0.0    ┆ 0.0    │
│ 5      ┆ 4.0 ┆ 0.0 ┆ 0.0 ┆ … ┆ 0.0    ┆ 0.0    ┆ 0.0    ┆ 0.0    │
│ …      ┆ …   ┆ …   ┆ …   ┆ … ┆ …      ┆ …      ┆ …      ┆ …      │
│ 606    ┆ 2.5 ┆ 0.0 ┆ 0.0 ┆ … ┆ 0.0    ┆ 0.0    ┆ 0.0    ┆ 0.0    │
│ 607    ┆ 4.0 ┆ 0.0 ┆ 0.0 ┆ … ┆ 0.0    ┆ 0.0    ┆ 0.0    ┆ 0.0    │
│ 608    ┆ 2.5 ┆ 2.0 ┆ 2.0 ┆ … ┆ 0.0    ┆ 0.0    ┆ 0.0    ┆ 0.0    │
│ 609    ┆ 3.0

In [163]:
# Combine movie rating columns into rating vector
rating_vectors = (
    ratings_matrix
    .with_columns(rating_vector=pl.concat_list(ratings_matrix.columns[1:]))
    .drop(pl.exclude(["userId", "rating_vector"]))
)
print(rating_vectors)

shape: (610, 2)
┌────────┬───────────────────┐
│ userId ┆ rating_vector     │
│ ---    ┆ ---               │
│ i32    ┆ list[f32]         │
╞════════╪═══════════════════╡
│ 1      ┆ [4.0, 0.0, … 0.0] │
│ 2      ┆ [0.0, 0.0, … 0.0] │
│ 3      ┆ [0.0, 0.0, … 0.0] │
│ 4      ┆ [0.0, 0.0, … 0.0] │
│ 5      ┆ [4.0, 0.0, … 0.0] │
│ …      ┆ …                 │
│ 606    ┆ [2.5, 0.0, … 0.0] │
│ 607    ┆ [4.0, 0.0, … 0.0] │
│ 608    ┆ [2.5, 2.0, … 0.0] │
│ 609    ┆ [3.0, 0.0, … 0.0] │
│ 610    ┆ [5.0, 0.0, … 0.0] │
└────────┴───────────────────┘


In [178]:
# Count similarity scores for userId 1 and all other users and take top 50 most similar users

def get_rating_vector_for_user(userId):
    return rating_vectors.filter(pl.col("userId") == userId).to_numpy()[0][1]


def get_top_n_most_similar_users(userId, rating_vector, n):
    return (
        rating_vectors
        .with_columns(
             pl.col("rating_vector")
             .map_elements(lambda x: get_pearson_correlation(rating_vector, x), return_dtype=pl.Float32)
             .alias("similarity_score")
        )
        .sort("similarity_score", descending=True)
        .filter(pl.col("userId") != userId)
        .limit(n)
        # Count additional information for prediction
        .with_columns(
            pl.col("rating_vector")
            # Calculate non-zero indices
            .map_elements(lambda x: np.nonzero(x)[0], return_dtype=pl.List(pl.Int32))
            .alias("rated_movie_indices")
        )
        .with_columns(
            pl.col("rating_vector")
            # Calculate mean of ratings, excluding zeros
            .map_elements(lambda x: np.sum(x.to_numpy()) / np.count_nonzero(x.to_numpy()), return_dtype=pl.Float32)
            .alias("ratings_mean")
        )
    )

userId = 1
query_user_rating_vector = get_rating_vector_for_user(userId)

# Get top 50 most similar users
top_50_most_similar = get_top_n_most_similar_users(userId, query_user_rating_vector, 50)
print(top_50_most_similar)

shape: (50, 5)
┌────────┬───────────────────┬──────────────────┬─────────────────────┬──────────────┐
│ userId ┆ rating_vector     ┆ similarity_score ┆ rated_movie_indices ┆ ratings_mean │
│ ---    ┆ ---               ┆ ---              ┆ ---                 ┆ ---          │
│ i32    ┆ list[f32]         ┆ f32              ┆ list[i32]           ┆ f32          │
╞════════╪═══════════════════╪══════════════════╪═════════════════════╪══════════════╡
│ 266    ┆ [2.0, 0.0, … 0.0] ┆ 0.344983         ┆ [0, 5, … 2450]      ┆ 3.5          │
│ 313    ┆ [0.0, 0.0, … 0.0] ┆ 0.333876         ┆ [5, 9, … 3937]      ┆ 3.432353     │
│ 368    ┆ [0.0, 0.0, … 0.0] ┆ 0.324041         ┆ [2, 5, … 3668]      ┆ 2.842217     │
│ 57     ┆ [5.0, 0.0, … 0.0] ┆ 0.323948         ┆ [0, 5, … 3668]      ┆ 3.392857     │
│ 39     ┆ [0.0, 0.0, … 0.0] ┆ 0.32012          ┆ [31, 43, … 2912]    ┆ 4.0          │
│ …      ┆ …                 ┆ …                ┆ …                   ┆ …            │
│ 307    ┆ [4.0, 2.5, … 0.0]

#### c)
Prediction function

In [165]:
def predict_movies(user_ratings_mean, similar_users_ratings, similar_users_ratings_mean, similarity_scores, candidate_movie_indices, num_of_movies):
    """
    The prediction function from course slides.

    :param user_ratings_mean: mean of user ratings
    :param similar_users_ratings: sparse numpy 2D matrix of similar users ratings. Each row is a user and each column is a movie
    :param similar_users_ratings_mean: numpy array of mean ratings of the similar users
    :param similarity_scores: numpy array of similarity scores between the user and the similar users
    :param candidate_movie_indices: numpy array of movie indices that the similar users have rated
    :param num_of_movies: number of movies to recommend
    :return: top ranked candidate movie indices, and their predicted ratings
    """

    # Compute ratings only for the candidate movies to save computation
    candidate_similar_users_ratings = np.take(similar_users_ratings, candidate_movie_indices, axis=1)

    # Subtract mean rating from each rating, (r_b,p - r_b)
    delta_ratings = candidate_similar_users_ratings - similar_users_ratings_mean

    # Sum sim(a,b) * (r_b,p - r_b)
    numerator = np.sum(similarity_scores.reshape(-1, 1) * delta_ratings, axis=0)

    # Sum |sim(a,b)|, taking absolute value to avoid negative bias even though it should not be possible
    denominator = np.sum(np.abs(similarity_scores))

    # Sum sim(a,b) * (r_b,p - r_b) / Sum |sim(a,b)|
    bias = numerator / denominator

    # pred(a,p) = r_a + bias
    predicted_ratings = user_ratings_mean + bias

    # Rank the movies by predicted rating
    indices = np.flip(np.argsort(predicted_ratings))
    predicted_movie_indices = candidate_movie_indices[indices][:num_of_movies]
    sorted_ratings = predicted_ratings[indices][:num_of_movies]

    return predicted_movie_indices, sorted_ratings

In [190]:
# Let's predict movies for userId 1 using the top 50 most similar users and the prediction function from the course slides

# Define some helper functions to extract data from the top n most similar users dataframe
def get_user_rating_mean(rating_vector):
    return np.mean(rating_vector[rating_vector != 0])

def get_get_similar_users_ratings_matrix(top_n_most_similar_users_df):
    return np.array(pl.Series(top_n_most_similar_users_df.select("rating_vector")).to_list())

def get_similar_users_ratings_mean(top_n_most_similar_users_df):
    return top_n_most_similar_users_df.select("ratings_mean").to_numpy()

def get_similar_users_similarity_scores(top_n_most_similar_users_df):
    return top_n_most_similar_users_df.select("similarity_score").to_numpy().reshape(-1)

def get_similar_users_most_watched_movies(top_n_most_similar_users_df, query_user_rated_movie_indices):
    return (
        top_n_most_similar_users_df
        .select(pl.col("rated_movie_indices").explode())
        .group_by("rated_movie_indices")
        .agg(
            pl.count("rated_movie_indices")
            .alias("ratings_count")
        )
        # Remove movies that the user has already seen
        .filter(pl.col("rated_movie_indices").is_in(query_user_rated_movie_indices).not_())
        # At least 3 users must have rated the movie
        .filter(pl.col("ratings_count") >= 3)
        .sort("ratings_count", descending=True)
    )

def get_candidate_movie_indices(similar_users_most_watched_movies):
    return similar_users_most_watched_movies.select("rated_movie_indices").to_numpy().reshape(-1)

query_user_rated_movie_indices = np.nonzero(query_user_rating_vector)[0]

similar_users_most_watched_movies = get_similar_users_most_watched_movies(top_50_most_similar, query_user_rated_movie_indices)
print(similar_users_most_watched_movies)

shape: (2_176, 2)
┌─────────────────────┬───────────────┐
│ rated_movie_indices ┆ ratings_count │
│ ---                 ┆ ---           │
│ i32                 ┆ u32           │
╞═════════════════════╪═══════════════╡
│ 507                 ┆ 46            │
│ 793                 ┆ 44            │
│ 659                 ┆ 41            │
│ 901                 ┆ 40            │
│ 2077                ┆ 40            │
│ …                   ┆ …             │
│ 1554                ┆ 3             │
│ 5365                ┆ 3             │
│ 3778                ┆ 3             │
│ 3138                ┆ 3             │
│ 5898                ┆ 3             │
└─────────────────────┴───────────────┘


In [191]:
# Recommend 10 movies for userId 1

user_ratings_mean = get_user_rating_mean(query_user_rating_vector)
similar_users_ratings = get_get_similar_users_ratings_matrix(top_50_most_similar)
similar_users_ratings_mean = get_similar_users_ratings_mean(top_50_most_similar)
similarity_scores = get_similar_users_similarity_scores(top_50_most_similar)
candidate_movie_indices = get_candidate_movie_indices(similar_users_most_watched_movies)

predicted_movies, ratings = predict_movies(user_ratings_mean, similar_users_ratings, similar_users_ratings_mean, similarity_scores, candidate_movie_indices, 10)

print('predicted movie indices:')
print(predicted_movies)
print()
print('predicted ratings:')
print(ratings)
print()

# print top 10 movies
print(movies_df.filter(pl.col("index").is_in(predicted_movies)))


predicted movie indices:
[ 507  659  901  793 2077   31  277  921 1157 1444]

predicted ratings:
[4.67787677 4.60761104 4.265487   4.25997751 4.07875882 3.96788518
 3.93025386 3.85082443 3.68561891 3.67868759]

shape: (10, 4)
┌───────┬─────────┬─────────────────────────────────┬─────────────────────────────────┐
│ index ┆ movieId ┆ title                           ┆ genres                          │
│ ---   ┆ ---     ┆ ---                             ┆ ---                             │
│ u32   ┆ i32     ┆ str                             ┆ str                             │
╞═══════╪═════════╪═════════════════════════════════╪═════════════════════════════════╡
│ 31    ┆ 32      ┆ Twelve Monkeys (a.k.a. 12 Monk… ┆ Mystery|Sci-Fi|Thriller         │
│ 277   ┆ 318     ┆ Shawshank Redemption, The (199… ┆ Crime|Drama                     │
│ 507   ┆ 589     ┆ Terminator 2: Judgment Day (19… ┆ Action|Sci-Fi                   │
│ 659   ┆ 858     ┆ Godfather, The (1972)           ┆ Crime|Drama     

#### d)

New similarity function

We chose the cosine similarity as a distance measure for the following reasons:
    - It is a common similarity measure for collaborative filtering
    - It is a good distance measure for sparse data

However, since the cosine similarity does not depend on magnitude of the vector we have to do some modifications to the rating scale. The ratings scale of 1-5 is shifted to -2-2 by subtracting 3 from each rating. This is done to make the cosine similarity work better with the data since otherwise the ratings of three movies [1, 1, 1] and [5, 5, 5] would be treated as eaqual in the cosine similarity calculation. By doing this shift the similarity measure will be more accurate in comparing user preferences of liking or disliking a movie while also ignoring neutral ratings.


In [ ]:
from scipy.spatial import distance

def cosine_similarity(a, b):
    # shift non-zero ratings to -2-2 scale
    a_mask = a != 0
    b_mask = b != 0
    a[a_mask] = a[a_mask] - 3
    b[b_mask] = b[b_mask] - 3

    # prevent division by zero
    if np.sum(a) == 0 or np.sum(b) == 0:
        return 0

    # cosine distance is 1 - cosine similarity
    return 1 - distance.cosine(a, b)


a = np.array([4, 4, 0, 4])
b = np.array([5, 0, 3, 5])
c = np.array([1, 1, 1, 0])
d = np.array([3, 2, 3, 2])

print(cosine_similarity(np.copy(a), np.copy(b)))
print(cosine_similarity(np.copy(a), np.copy(c)))
print(cosine_similarity(np.copy(a), np.copy(d)))
print(cosine_similarity(np.copy(c), np.copy(d)))

#### e)
User-based collaborative filtering for group recommendations

In [ ]:
# Create a group of users by picking the first 5 userIds
group = rating_vectors.filter(pl.col("userId").is_in([1, 2, 3, 4, 5]))
print(group)